In [1]:
from src.crawlers import DataCrawlers, VendorSubCategoryCrawler
from src.utils import concat_data, get_file_list, get_latest_session_index
from ipywidgets import Layout, HBox
import ipywidgets as widgets
from IPython.display import display
import os

driver_path = 'src/geckodriver.exe'

btn_start = widgets.Button(
    description='Start Crawl',
    button_style='success',
)
btn_combine_data = widgets.Button(
    description='Combine Data',
    button_style='success',
)
tb_url = widgets.Text(
    placeholder='Enter Vendor URL here',
    description='Vendor URL:', 
    layout=Layout(width='85%')
)

dk_data_dir = os.path.join(os.getcwd(), 'DK_Data_By_Vendor')
os.makedirs(dk_data_dir, exist_ok=True)

display(HBox([btn_start, btn_combine_data]), tb_url)

def urls_by_vendor(vendor_url):
    vendor_crawler = VendorSubCategoryCrawler(driver_path, vendor_url, 
                                              download_dir=None)
    subcategory_urls = vendor_crawler.crawl()
    vendor_crawler.close()
    return subcategory_urls

def start_crawl(b):
    vendor_url = tb_url.value
    vendor = vendor_url.split('/')[-1]
    download_dir = os.path.join(dk_data_dir, vendor)
    os.makedirs(download_dir, exist_ok=True)
    
    subcategory_urls = urls_by_vendor(vendor_url)
    data_crawlers = DataCrawlers(driver_path, subcategory_urls, download_dir, 
                                 n_workers=3, headless=True)
    data_crawlers.crawl_all()

def combine_subcat_data(b):
    vendor_url = tb_url.value
    vendor = vendor_url.split('/')[-1]
    download_dir = os.path.join(dk_data_dir, vendor)
    
    in_files = get_file_list(download_dir, suffix='all.xlsx')
    out_path = os.path.join(download_dir, 'combine.xlsx')
    df = concat_data(in_files)
    df.to_excel(out_path)
    print(f'Data combined at {out_path}')

btn_start.on_click(start_crawl)
btn_combine_data.on_click(combine_subcat_data)

Text(value='', description='Vendor URL:', layout=Layout(width='85%'), placeholder='Enter Vendor URL here')